<h1 align="center"> Implementation of two algorithms for computing the general component of jet scheme in SageMath </h1>

<h2 align="center"> Mario Morán Cañón and Julien Sebag </h2>

<h3 align="center"> Supplementary material to the article <br/><br/><i>Two algorithms for computing the general component of jet scheme and applications</i>, Mario Morán Cañón and Julien Sebag, J. Symbolic Comput 113 (2022), 74-96. </h3>

This file contains an implementation of the algorithms developed in the article, with a description of each function. We also provide these functions in the file `algorithms_general_component.sage` for the ease of the reader interested in using them.

Examples of use of these algorithms are available in the file `Examples of computation of the general component of the jet scheme in SageMath.ipynb`.

Function `jet_ring`: constructs the section ring of the jet scheme of some level of an affine scheme.

**Inputs:**

- `base`: the base field.
- `var`: a list $[v_0,\dots,v_n]$ of strings containing the names of the variables of the (polynomial) section ring of the affine space.
- `dim`: a list $[d_0,\dots,d_n]$ of positive integers of the same lenght as `var`. If some integer $d_i$ is greater than 1, then the base polynomial ring is supposed to have $d_i$ copies of the variable $v_i$, which are called $v_{i,0},\dots,v_{i,d_i-1}$.
- `trun`: a positive integer, the level of the jet scheme plus 1.
- `monomial_order`: the monomial order on the section ring of the jet scheme, described with respect to the ordered list of variables $v_{0,0},\dots,v_{0,d_0-1},v_{1,0},\dots,v_{1,d_1-1},\dots,v_{n,0},\dots,v_{n,d_n-1}$.

**Output:** the (polynomial) section ring of the jet scheme of level `trun`-1 of the affine scheme whose section ring is the polynomial ring in the variables in `var`, each one appearing as many times as the corresponding integer in `dim`.

In [1]:
def jet_ring(base,var,dim,trun,monomial_order):
    variables=[];
    for r in range(len(var)):
        if dim[r]==1:
            variables.extend([var[r]+'%i' %i for i in range(trun)]);
        else:
            for j in range(dim[r]):
                variables.extend([var[r]+str(j)+'%i' %i for i in range(trun)]);
    R=PolynomialRing(base, variables, order=monomial_order);
    return R

For example, in order to construct the section ring $\mathbb{Q}[x,y,T_0,T_1,T_2]_5$ of $\mathcal{L}_{5}(\mathrm{Spec}(\mathbb{Q}[x,y,T_0,T_1,T_2]))$ with the lexicographic order $T_{2,5}>\dots>T_{2,0}>T_{1,5}>\dots T_{1,0}>T_{0,5}>\dots T_{0,0}>y_{5}>\dots y_{0}>x_{5}>\dots x_{0}$ we write:

In [2]:
jet_ring(QQ,['x','y','T'],[1,1,3],6,'invlex')

Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, y0, y1, y2, y3, y4, y5, T00, T01, T02, T03, T04, T05, T10, T11, T12, T13, T14, T15, T20, T21, T22, T23, T24, T25 over Rational Field

Function `truncation_order`: given a (polynomial) section ring of a jet scheme of level $m$ of an affine space, returns the order of truncation $m+1$.

**Inputs:**

- `R`: a (polynomial) section ring of a jet scheme of level $m$ of an affine space. It must be of the form of the ones constructed using the function `jet_ring`.

**Output:** the order of truncation $m+1$.

In [3]:
def truncation_order(R):
    c=len(R.variable_names()[-1])-len(R.variable_names()[0])+1;
    trun=int(R.variable_names()[-1][-c:])+1;
    return trun

Function `HasseSchmidt`: computes the polynomials in the ideal defining the jet scheme of a given level from the polynomials defining the base variety.

**Inputs:**

- `L`: a list $L=[f_1,\dots,f_r]$ of elements of $A_0$, the section ring of the $0$-jet scheme (which is canonically isomorphic to $A$, the (polynomial) section ring of the base affine scheme.
- `N`: a non-negative integer number.

**Output:** A list of lists $[[f_{1,0},\dots,f_{1,N}],\dots, [f_{r,0},\dots,f_{r,N}]]$, such that if $A=k[x_1,\dots,x_n]$ then for every $1\le i\le s$ we have the following equality in $k[x_1,\dots,x_n]_N[t]$:
\\[f_i\left(\left(\sum\limits_{j=0}^{N}x_{i,j}t^{j}\right)_{0\le i\le n}\right)=\sum\limits_{s=0}^{N}f_{i,s}\left((x_{i,j})_{\substack{0\le i\le n\\0\le j\le s}}\right)t^s \hspace{5mm}(\mathrm{mod}\; t^{N+1})\\]

In [4]:
def HasseSchmidt (L,N):
    trun=truncation_order(L[0].parent());
    v=list(L[0].parent().variable_names());
    v.append('t');
    nvar=L[0].parent().ngens()//trun;
    R1=PolynomialRing(L[0].parent().base_ring(), v, order=L[0].parent().term_order());
    varser=[sum([R1.gen(r*trun+i)*R1.gen(R1.ngens()-1)^i for i in range(N+1)]) for r in range(nvar)];
    dicsub = dict((R1.gen(r*trun),varser[r]) for r in range(nvar));
    Lseries=[R1(s).subs(dicsub) for s in L];
    LHS=[[L[0].parent()(s.coefficient({R1.gen(R1.ngens()-1):i})) for i in range(N+1)] for s in Lseries];
    return LHS

Function `HS_to_ideal`: returns the ideal generated by the polynomials in the input of `HasseSchmidt`.

**Inputs:**

- `LHS`: a list of lists $[[f_{1,0},\dots,f_{1,N}],\dots, [f_{r,0},\dots,f_{r,N}]]$ as in the output of `HasseSchmidt`.
- `N`: a non-negative integer number, corresponding to the size of any of the inner lists in `LHS`.

**Output:** The ideal generated by all the polynomials in `LHS`.

In [5]:
def HS_to_ideal (LHS,N):
    gen=[];
    for s in LHS:
        gen=gen + s[:N+1];
    HSideal=gen[0].parent().ideal(gen);
    return HSideal    

Function `jet_ring_HS`: given a polynomial ring and an ideal of this ring defining a variety $V$, it computes the section ring of the jet scheme of a given level `trun`=$m+1$ of the corresponding affine scheme and the ideal defining $\mathcal{L}_{m}(V)$ as a closed subscheme.

**Inputs:**

- `base_affine`: a polynomial ring, the section ring of the base affine scheme.
- `base_ideal`: an ideal of `base_affine` defining the base variety $V$.
- `trun`: a positive integer , the level $m$ of the jet scheme plus 1.

**Output:** A list containing:
- In the first position the (polynomial) section ring of the jet scheme of level `trun`-1 of the affine scheme corresponding to `base_affine`.
- In the second position the ideal of the preceding polynomial ideal defining $\mathcal{L}_{m}(V)$.

In [6]:
def jet_ring_HS(base_affine, base_ideal, trun):
    var=list(base_affine.variable_names());
    R=jet_ring(base_affine.base_ring(),var,[1 for r in var],trun,base_affine.term_order());
    inj=base_affine.hom([R.gen(i*trun) for i in range(base_affine.ngens())]);
    gen_base_ideal=base_ideal.gens();
    inj_base_ideal=[inj(g) for g in gen_base_ideal];
    HSideal_gen=HasseSchmidt(inj_base_ideal,trun-1);
    HSideal=HS_to_ideal(HSideal_gen,trun-1);
    return [R,HSideal]

Function `base_ring_inclusion_jet_ring`: given the section ring $A$ of a base affine scheme and the section ring $A_m$ of its jet scheme of some level, it constructs the inclusion morphism $A\rightarrow A_m$.

**Inputs:**

- `base_affine`: a polynomial ring, the section ring $A$ of the base affine scheme.
- `R`: a polynomial ring, the section ring $A_m$ of the jet scheme of some level of the affine scheme corresponding to `base_affine`. It must be of the form of the rings contructed using `jet_ring`.

**Output:** The inclusion morphism $A\rightarrow A_m$.

In [7]:
def base_ring_inclusion_jet_ring(base_affine,R):
    trun=truncation_order(R);
    phi=base_affine.hom([R.gen(trun*i) for i in range(base_affine.ngens())])
    return phi

Function `jet_rings_canonical`: given two jet schemes $A_m$ and $A_n$ of the same base affine scheme with $m<n$, it constructs the canonical morphism $A_m\rightarrow A_n$.

**Inputs:**

- `small_jet_ring`: a polynomial ring, the section ring $A_m$ of the jet scheme of level $m$ of some affine scheme. It must be of the form of the rings constructed using `jet_ring`.
- `big_jet_ring`: a polynomial ring, the section ring $A_n$ of the jet scheme of level $n>m$ of some affine scheme. It must be of the form of the rings constructed using `jet_ring`.

**Output:** The canonical morphism $A_m\rightarrow A_n$.

In [18]:
def jet_rings_canonical(small_jet_ring,big_jet_ring):
    small_trun=truncation_order(small_jet_ring);
    big_trun=truncation_order(big_jet_ring);
    nvar=small_jet_ring.ngens()//small_trun;
    inj_images=[];
    for r in range(nvar):
        inj_images.extend([big_jet_ring.gen(r*big_trun+i) for i in range(small_trun)]);
    jet_inj=small_jet_ring.hom(inj_images);
    return jet_inj

**Algorithm 1**

Function `general_component_saturation`: given a polynomial ring and an ideal of this ring defining a variety $V$, it computes the ideal $\mathcal{N}_N(V)$ defining the general component of the jet scheme of level $N$ of $V$.

**Inputs:**

- `base_affine`: a polynomial ring, the section ring of the base affine scheme.
- `base_ideal`: an ideal of `base_affine` defining the base variety $V$.
- `N`: a positive integer, the level of the jet scheme.

**Output:** The ideal $\mathcal{N}_N(V)$ defining the general component of $\mathcal{L}_{N}(V)$.

In [8]:
def general_component_saturation(base_affine,base_ideal,N):
    # We construct the affine variety, compute the Jacobian ideal and choose the element H for the saturation
    affine_space=AffineSpace(base_affine);
    variety=affine_space.subscheme(base_ideal);
    jac_ideal=variety.Jacobian();
    jac_generators=jac_ideal.gens();
    for g in jac_generators:
        if base_ideal.reduce(g)!=0:
            H=g;
            break
    # We construct the jet ring and the ideal of Hasse-Schmidt derivatives and inject the chosen H
    [jetring,ideal_HS]=jet_ring_HS(base_affine, base_ideal, N+1);
    base_inj_jet=base_ring_inclusion_jet_ring(base_affine,jetring);
    H_jet=base_inj_jet(H);
    # We extend the ring for the computation of the saturation as an elimination ideal
    v=list(jetring.variable_names());
    v.append('t');
    R1=PolynomialRing(jetring.base_ring(), v, order=jetring.term_order());
    # We define the inclusion morphism from jetring in R1 and compute the extension of the ideal
    inj_extension=jetring.hom(list(R1.gens())[:-1]);
    ideal_HS_extension=inj_extension(ideal_HS);
    # We construct the aimed ideal and perform the elimination
    sat_element=R1.ideal([1-inj_extension(H_jet)*R1.gen(R1.ngens()-1)]);
    sat_ideal=ideal_HS_extension+sat_element;
    elim_ideal=sat_ideal.elimination_ideal([R1.gen(R1.ngens()-1)]);
    # We change the resulting ideal to the original ring
    contraction_morph=R1.hom(list(jetring.gens())+[0]);
    saturation=contraction_morph(elim_ideal);
    return saturation

**Algorithm 2**

Function `general_component_saturation`: given a smooth birational model of a variety $V$, it computes the ideal $\mathcal{N}_N(V)$ defining the general component of the jet scheme of level $N$ of $V$.

**Inputs:**

- `base_affine`: a polynomial ring, the section ring of the base affine scheme.
- `birring`: a polynomial ring, the section ring of the affine scheme which the smooth birational model is a closed subscheme of.
- `birideal`: a list of elements of `birring` which generate the ideal defining the smooth birational model.
- `birimage`: a list of elements of `birring` which contain the images of the variables in `base_affine` via the birational morphism.
- `N`: a positive integer, the level of the jet scheme.

**Output:** The ideal $\mathcal{N}_N(V)$ defining the general component of $\mathcal{L}_{N}(V)$.

In [9]:
def general_component_birational(base_affine,birring,birideal,birimage,N):
    # We construct the corresponding jet ring
    var=list(base_affine.variable_names());
    R=jet_ring(base_affine.base_ring(),var,[1 for r in var],N+1,base_affine.term_order());
    trun=N+1;
    nvar=base_affine.ngens();
    # We construct the enlarged ring
    varbir=list(birring.variable_names());
    varbase=var;
    varaug=varbase+varbir;
    dimaug=[1 for s in varaug];
    R_aug=jet_ring(R.base_ring(),varaug,dimaug,trun,R.term_order());
    # We define the ring extension of the ring of the birational model into the elarged ring
    bir_inj=birring.hom([R_aug.gen((nvar + i)*trun) for i in range(birring.ngens())]);
    # We define the extension in the enlarged ring of the ideal defining the jet scheme of the birational model via HS
    gen_birideal_inj=[bir_inj(r) for r in birideal];
    gen_birideal_aug_HS=HasseSchmidt(gen_birideal_inj,N);
    birideal_aug_HS=HS_to_ideal(gen_birideal_aug_HS,N);
    # We define the ideal of the enlarged ring generated by the elements corresponding to the images of the variables via the birational morphism
    birimage_inj=[bir_inj(r) for r in birimage];
    dif_image=[R_aug.gen(trun*i)-birimage_inj[i] for i in range(len(birimage_inj))];
    dif_image_HS=HasseSchmidt(dif_image,N);
    ideal_image_HS=HS_to_ideal(dif_image_HS,N);
    # We define the sum of the preceding ideals and perform the elimination of variables to obtain the kernel
    K=ideal_image_HS + birideal_aug_HS;
    ker=K.elimination_ideal([R_aug.gen(j) for j in range(nvar*trun,len(varaug)*trun)])
    # We consider the obtained ideal in the original ring
    gen_ker=ker.gens();
    gen_ker_original_ring=[R(g) for g in gen_ker];
    ker_original=R.ideal(gen_ker_original_ring);
    return ker_original